In [1]:
%%writefile ex.txt
first
second line
the third line
then a fourth line

Overwriting ex.txt


In [2]:
from pyspark import SparkContext
sc = SparkContext()

In [5]:
text_rdd = sc.textFile('ex.txt')
text_rdd.collect()

['first', 'second line', 'the third line', 'then a fourth line']

In [4]:
words = text_rdd.map(lambda line: line.split())
words.collect()

[['first'],
 ['second', 'line'],
 ['the', 'third', 'line'],
 ['then', 'a', 'fourth', 'line']]

In [6]:
text_rdd.flatMap(lambda line: line.split()).collect()

['first',
 'second',
 'line',
 'the',
 'third',
 'line',
 'then',
 'a',
 'fourth',
 'line']

# rdd key-value pairs

In [7]:
%%writefile services.txt
#EventId    Timestamp    Customer   State    ServiceID    Amount
201       10/13/2017      100       NY       131          100.00
204       10/18/2017      700       TX       129          450.00
202       10/15/2017      203       CA       121          200.00
206       10/19/2017      202       CA       131          500.00
203       10/17/2017      101       NY       173          750.00
205       10/19/2017      202       TX       121          200.00

Writing services.txt


In [9]:
services = sc.textFile('services.txt')
services.collect()

['#EventId    Timestamp    Customer   State    ServiceID    Amount',
 '201       10/13/2017      100       NY       131          100.00',
 '204       10/18/2017      700       TX       129          450.00',
 '202       10/15/2017      203       CA       121          200.00',
 '206       10/19/2017      202       CA       131          500.00',
 '203       10/17/2017      101       NY       173          750.00',
 '205       10/19/2017      202       TX       121          200.00']

In [10]:
services.take(2)

['#EventId    Timestamp    Customer   State    ServiceID    Amount',
 '201       10/13/2017      100       NY       131          100.00']

In [11]:
services.map(lambda line: line.split()).take(3)

[['#EventId', 'Timestamp', 'Customer', 'State', 'ServiceID', 'Amount'],
 ['201', '10/13/2017', '100', 'NY', '131', '100.00'],
 ['204', '10/18/2017', '700', 'TX', '129', '450.00']]

In [14]:
clean_rdd = services.map(lambda line: line[1:] if line[0]=='#' else line)
clean_rdd = clean_rdd.map(lambda line: line.split())

clean_rdd.collect()

[['EventId', 'Timestamp', 'Customer', 'State', 'ServiceID', 'Amount'],
 ['201', '10/13/2017', '100', 'NY', '131', '100.00'],
 ['204', '10/18/2017', '700', 'TX', '129', '450.00'],
 ['202', '10/15/2017', '203', 'CA', '121', '200.00'],
 ['206', '10/19/2017', '202', 'CA', '131', '500.00'],
 ['203', '10/17/2017', '101', 'NY', '173', '750.00'],
 ['205', '10/19/2017', '202', 'TX', '121', '200.00']]

In [20]:
pairs = clean_rdd.map(lambda l: (l[3],l[5]))
reduce_keys = pairs.reduceByKey(lambda amt1, amt2: float(amt1) + float(amt2))

reduce_keys.collect()

[('State', 'Amount'), ('NY', 850.0), ('TX', 650.0), ('CA', 700.0)]

In [21]:
no_headers = reduce_keys.filter(lambda x: not x[0]=='State')
final_sorted = no_headers.sortBy(lambda stAmount: stAmount[1], ascending=False)
final_sorted.collect()

[('NY', 850.0), ('CA', 700.0), ('TX', 650.0)]

In [22]:
x = ['ID', 'State', 'Amount']

In [24]:
def func1(l):
    return l[-1]

#test
func1(x)

'Amount'

In [25]:
def func2(id_st_amt):
    #unpack values
    (id_, st, amt) = id_st_amt
    return amt

#test
func2(x)

'Amount'